# Dokumentasi dan Kesimpulan Penelitian

## Analisis Sentimen Twitter Mengenai Sertifikasi Halal

**Metode:** K-Means Clustering dan Naive Bayes Classification

---

Notebook ini berisi:
1. Ringkasan Statistik Deskriptif
2. Parameter yang Digunakan
3. Perbandingan Hasil K-Means dan Naive Bayes
4. Kelebihan dan Keterbatasan Metode
5. Kesimpulan Penelitian

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load data hasil akhir
try:
    df = pd.read_csv("data/hasil_klasifikasi.csv")
    print("Data berhasil dimuat dari hasil_klasifikasi.csv")
except:
    try:
        df = pd.read_csv("data/hasil_clustering.csv")
        print("Data berhasil dimuat dari hasil_clustering.csv")
    except:
        df = pd.read_csv("data/dataSertifikasiHalal.csv")
        print("Data berhasil dimuat dari dataSertifikasiHalal.csv")

print(f"\nJumlah data: {len(df)}")

---

## 1. Ringkasan Statistik Deskriptif

In [ ]:
print("="*70)
print("RINGKASAN STATISTIK DESKRIPTIF DATASET")
print("="*70)

In [ ]:
# Informasi umum dataset
print("\n1.1 INFORMASI UMUM DATASET")
print("-"*40)
print(f"Jumlah total data (tweet): {len(df)}")
print(f"Jumlah kolom: {len(df.columns)}")
print(f"Kolom yang tersedia: {df.columns.tolist()}")

In [ ]:
# Distribusi sentimen
print("\n1.2 DISTRIBUSI SENTIMEN")
print("-"*40)

if 'sentiment' in df.columns:
    print("\nDistribusi Sentimen (K-Means Clustering):")
    sentiment_dist = df['sentiment'].value_counts()
    for sentiment, count in sentiment_dist.items():
        pct = count / len(df) * 100
        print(f"  {sentiment.capitalize():10} : {count:5} data ({pct:5.2f}%)")

if 'predicted_sentiment' in df.columns:
    print("\nDistribusi Sentimen (Naive Bayes Prediction):")
    pred_dist = df['predicted_sentiment'].value_counts()
    for sentiment, count in pred_dist.items():
        pct = count / len(df) * 100
        print(f"  {sentiment.capitalize():10} : {count:5} data ({pct:5.2f}%)")

In [ ]:
# Statistik panjang teks
print("\n1.3 STATISTIK PANJANG TEKS")
print("-"*40)

text_col = 'preprocessed_text' if 'preprocessed_text' in df.columns else 'cleaned_text' if 'cleaned_text' in df.columns else 'full_text'

if text_col in df.columns:
    df['char_count'] = df[text_col].str.len()
    df['word_count'] = df[text_col].str.split().str.len()
    
    print(f"\nStatistik Jumlah Karakter ({text_col}):")
    print(f"  Minimum  : {df['char_count'].min():.0f}")
    print(f"  Maksimum : {df['char_count'].max():.0f}")
    print(f"  Rata-rata: {df['char_count'].mean():.2f}")
    print(f"  Median   : {df['char_count'].median():.0f}")
    print(f"  Std Dev  : {df['char_count'].std():.2f}")
    
    print(f"\nStatistik Jumlah Kata ({text_col}):")
    print(f"  Minimum  : {df['word_count'].min():.0f}")
    print(f"  Maksimum : {df['word_count'].max():.0f}")
    print(f"  Rata-rata: {df['word_count'].mean():.2f}")
    print(f"  Median   : {df['word_count'].median():.0f}")
    print(f"  Std Dev  : {df['word_count'].std():.2f}")

In [ ]:
# Statistik per sentimen
print("\n1.4 STATISTIK PER SENTIMEN")
print("-"*40)

if 'sentiment' in df.columns and 'word_count' in df.columns:
    print("\nRata-rata jumlah kata per sentimen:")
    for sentiment in df['sentiment'].unique():
        avg_words = df[df['sentiment'] == sentiment]['word_count'].mean()
        print(f"  {sentiment.capitalize():10} : {avg_words:.2f} kata")

---

## 2. Parameter yang Digunakan

In [ ]:
print("="*70)
print("PARAMETER YANG DIGUNAKAN")
print("="*70)

In [ ]:
print("\n2.1 PARAMETER DATA CLEANING")
print("-"*40)
print("""
Tahapan cleaning yang dilakukan:
  1. Hapus URL (http://, https://, www., bit.ly, t.co)
  2. Hapus Mention (@username)
  3. Hapus Hashtag (#topic)
  4. Hapus Emoji dan karakter Unicode khusus
  5. Hapus Angka
  6. Hapus Karakter khusus (hanya menyisakan huruf dan spasi)
  7. Hapus Whitespace berlebih
  8. Hapus Duplikat berdasarkan teks yang sudah dibersihkan
""")

In [ ]:
print("\n2.2 PARAMETER TEXT PREPROCESSING")
print("-"*40)
print("""
Tahapan preprocessing yang dilakukan:
  1. Case Folding: Mengubah semua huruf menjadi lowercase
  2. Cleaning: Menghapus simbol, angka, dan karakter non-huruf
  3. Tokenizing: Memecah teks menjadi kata-kata (token)
     - Filter kata dengan panjang <= 2 karakter
  4. Stopword Removal: Menghapus kata-kata umum bahasa Indonesia
     - Menggunakan daftar stopword kustom (150+ kata)
     - Termasuk slang Twitter (yg, dgn, utk, gak, dll)
  
  CATATAN: Stemming TIDAK digunakan untuk menjaga kata tetap utuh
  (contoh: 'sertifikasi' tidak diubah menjadi 'rtifikas')
""")

In [ ]:
print("\n2.3 PARAMETER TF-IDF VECTORIZATION")
print("-"*40)
print("""
Parameter TfidfVectorizer:
  - max_features: 1000 (membatasi jumlah fitur/kata)
  - min_df: 2 (kata minimal muncul di 2 dokumen)
  - max_df: 0.95 (kata maksimal muncul di 95% dokumen)
""")

In [ ]:
print("\n2.4 PARAMETER K-MEANS CLUSTERING")
print("-"*40)
print("""
Parameter KMeans:
  - n_clusters: 3 (positif, negatif, netral)
  - random_state: 42 (untuk reprodusibilitas)
  - n_init: 10 (jumlah inisialisasi centroid)
  - max_iter: 300 (maksimum iterasi)
  
Metode penentuan k optimal:
  - Elbow Method (range k=2 sampai k=10)
  - Silhouette Score
""")

In [ ]:
print("\n2.5 PARAMETER NAIVE BAYES CLASSIFICATION")
print("-"*40)
print("""
Parameter MultinomialNB:
  - alpha: 1.0 (Laplace smoothing)
  
Parameter Train-Test Split:
  - test_size: 0.2 (20% untuk testing)
  - random_state: 42 (untuk reprodusibilitas)
  - stratify: y (menjaga proporsi kelas)
""")

---

## 3. Perbandingan Hasil K-Means dan Naive Bayes

In [ ]:
print("="*70)
print("PERBANDINGAN HASIL K-MEANS DAN NAIVE BAYES")
print("="*70)

In [ ]:
if 'sentiment' in df.columns and 'predicted_sentiment' in df.columns:
    print("\n3.1 TABEL PERBANDINGAN DISTRIBUSI")
    print("-"*40)
    
    comparison_df = pd.DataFrame({
        'K-Means': df['sentiment'].value_counts(),
        'Naive Bayes': df['predicted_sentiment'].value_counts()
    })
    comparison_df['Selisih'] = comparison_df['K-Means'] - comparison_df['Naive Bayes']
    print(comparison_df)

In [ ]:
if 'sentiment' in df.columns and 'predicted_sentiment' in df.columns:
    print("\n3.2 KESESUAIAN LABEL")
    print("-"*40)
    
    # Hitung kesesuaian
    match_count = (df['sentiment'] == df['predicted_sentiment']).sum()
    mismatch_count = len(df) - match_count
    match_pct = match_count / len(df) * 100
    
    print(f"Data dengan label sama    : {match_count} ({match_pct:.2f}%)")
    print(f"Data dengan label berbeda : {mismatch_count} ({100-match_pct:.2f}%)")

In [ ]:
if 'sentiment' in df.columns and 'predicted_sentiment' in df.columns:
    print("\n3.3 CROSS-TABULATION")
    print("-"*40)
    
    crosstab = pd.crosstab(
        df['sentiment'], 
        df['predicted_sentiment'],
        margins=True,
        margins_name='Total'
    )
    print("\nTabel silang K-Means vs Naive Bayes:")
    print(crosstab)

In [ ]:
print("\n3.4 ANALISIS KESESUAIAN")
print("-"*40)
print("""
Interpretasi:
- Kesesuaian tinggi menunjukkan bahwa hasil clustering K-Means
  dapat digunakan sebagai label yang valid untuk klasifikasi.
- Perbedaan label dapat terjadi karena:
  1. Ambiguitas teks yang sulit dikategorikan
  2. Keterbatasan K-Means dalam menangkap nuansa sentimen
  3. Naive Bayes mempelajari pola dari data training
""")

---

## 4. Kelebihan dan Keterbatasan Metode

In [ ]:
print("="*70)
print("KELEBIHAN DAN KETERBATASAN METODE")
print("="*70)

In [ ]:
print("\n4.1 K-MEANS CLUSTERING")
print("-"*40)
print("""
KELEBIHAN:
  ✓ Tidak memerlukan data berlabel (unsupervised)
  ✓ Dapat menemukan pola tersembunyi dalam data
  ✓ Komputasi relatif cepat untuk dataset besar
  ✓ Mudah diimplementasikan dan diinterpretasikan
  ✓ Cocok untuk eksplorasi awal data

KETERBATASAN:
  ✗ Harus menentukan jumlah cluster (k) di awal
  ✗ Sensitif terhadap inisialisasi centroid
  ✗ Tidak optimal untuk cluster non-spherical
  ✗ Memerlukan interpretasi manual untuk labeling sentimen
  ✗ Tidak mempertimbangkan urutan kata (bag-of-words)
""")

In [ ]:
print("\n4.2 NAIVE BAYES CLASSIFICATION")
print("-"*40)
print("""
KELEBIHAN:
  ✓ Cepat dalam training dan prediksi
  ✓ Bekerja baik dengan data teks (high-dimensional)
  ✓ Tidak memerlukan banyak data training
  ✓ Memberikan probabilitas untuk setiap kelas
  ✓ Robust terhadap fitur yang tidak relevan

KETERBATASAN:
  ✗ Asumsi independensi antar fitur (naive)
  ✗ Memerlukan data berlabel untuk training
  ✗ Kualitas prediksi bergantung pada kualitas label
  ✗ Tidak menangkap hubungan antar kata
  ✗ Sensitif terhadap ketidakseimbangan kelas
""")

In [ ]:
print("\n4.3 TF-IDF VECTORIZATION")
print("-"*40)
print("""
KELEBIHAN:
  ✓ Memberikan bobot lebih pada kata penting
  ✓ Mengurangi pengaruh kata umum
  ✓ Representasi numerik yang efektif

KETERBATASAN:
  ✗ Tidak mempertimbangkan urutan kata
  ✗ Tidak menangkap makna semantik
  ✗ Sparse matrix untuk vocabulary besar
""")

---

## 5. Kesimpulan Penelitian

In [ ]:
print("="*70)
print("KESIMPULAN PENELITIAN")
print("="*70)

In [ ]:
# Tentukan sentimen dominan
if 'sentiment' in df.columns:
    dominant_sentiment = df['sentiment'].value_counts().idxmax()
    dominant_count = df['sentiment'].value_counts().max()
    dominant_pct = dominant_count / len(df) * 100
    
    print(f"\n5.1 SENTIMEN DOMINAN MASYARAKAT")
    print("-"*40)
    print(f"\nBerdasarkan analisis {len(df)} tweet tentang sertifikasi halal:")
    print(f"\nSentimen dominan: {dominant_sentiment.upper()}")
    print(f"Jumlah: {dominant_count} tweet ({dominant_pct:.2f}%)")

In [ ]:
print("\n5.2 RINGKASAN TEMUAN")
print("-"*40)

if 'sentiment' in df.columns:
    sentiment_summary = df['sentiment'].value_counts()
    
    print("\nDistribusi sentimen masyarakat terhadap sertifikasi halal:")
    for sentiment in ['positif', 'negatif', 'netral']:
        if sentiment in sentiment_summary.index:
            count = sentiment_summary[sentiment]
            pct = count / len(df) * 100
            print(f"  - {sentiment.capitalize():8}: {pct:.1f}%")

In [ ]:
print("\n5.3 INTERPRETASI HASIL")
print("-"*40)
print("""
Berdasarkan hasil analisis sentimen menggunakan kombinasi metode
K-Means Clustering dan Naive Bayes Classification:

1. PENDEKATAN HYBRID:
   - K-Means digunakan untuk eksplorasi awal dan labeling otomatis
   - Naive Bayes digunakan untuk klasifikasi dan validasi
   - Kombinasi kedua metode memberikan hasil yang lebih robust

2. KUALITAS CLUSTERING:
   - Silhouette Score menunjukkan kualitas pemisahan cluster
   - Elbow Method membantu menentukan jumlah cluster optimal
   - k=3 dipilih untuk merepresentasikan sentimen positif, negatif, netral

3. PERFORMA KLASIFIKASI:
   - Model Naive Bayes dapat memprediksi sentimen dengan akurasi yang baik
   - Kesesuaian dengan label K-Means menunjukkan konsistensi hasil
""")

In [ ]:
print("\n5.4 SARAN UNTUK PENELITIAN SELANJUTNYA")
print("-"*40)
print("""
1. PENINGKATAN PREPROCESSING:
   - Menambahkan normalisasi kata tidak baku (slang)
   - Menggunakan kamus sentimen bahasa Indonesia
   - Menangani negasi (tidak, bukan, jangan)

2. METODE ALTERNATIF:
   - Mencoba algoritma deep learning (LSTM, BERT)
   - Menggunakan word embedding (Word2Vec, FastText)
   - Menerapkan ensemble methods

3. VALIDASI LEBIH LANJUT:
   - Melakukan labeling manual untuk validasi
   - Cross-validation untuk evaluasi yang lebih robust
   - Analisis error untuk memahami kesalahan prediksi

4. EKSPANSI DATA:
   - Mengumpulkan data dari periode waktu yang lebih panjang
   - Menambahkan data dari platform media sosial lain
   - Mempertimbangkan konteks temporal (tren waktu)
""")

---

## Ringkasan Akhir

In [ ]:
print("\n" + "="*70)
print("RINGKASAN AKHIR PENELITIAN")
print("="*70)

print(f"""
Judul: Analisis Sentimen Twitter Mengenai Sertifikasi Halal
       Menggunakan K-Means Clustering dan Naive Bayes Classification

Dataset:
  - Sumber: Twitter
  - Jumlah data: {len(df)} tweet
  - Topik: Sertifikasi Halal di Indonesia

Metodologi:
  1. Data Collection (Scraping)
  2. Data Cleaning
  3. Text Preprocessing
  4. Feature Extraction (TF-IDF)
  5. Clustering (K-Means)
  6. Classification (Naive Bayes)
  7. Visualisasi dan Evaluasi

Hasil:
  - Sentimen berhasil diklasifikasikan ke dalam 3 kategori
  - Model dapat digunakan untuk prediksi sentimen baru
  - Visualisasi membantu interpretasi hasil
""")

if 'sentiment' in df.columns:
    print("Distribusi Sentimen Final:")
    for sentiment, count in df['sentiment'].value_counts().items():
        pct = count / len(df) * 100
        print(f"  - {sentiment.capitalize():8}: {count:5} ({pct:5.1f}%)")

print("\n" + "="*70)